In [3]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.models import Sequential, load_model
from sklearn.preprocessing import MinMaxScaler
import pandas as pd
import numpy as np
import os
from tqdm import tqdm  # 导入 tqdm

# 读取训练好的模型
model = load_model('predict_model.h5')

# 读取需要预测的数据
Import_csv_file = './merged_output.csv'
Import_df = pd.read_csv(Import_csv_file)

# 删除重复的 'Time' 列，假设 'Time' 列在第一列和第七列都有
Import_df.drop(columns=['Time'], inplace=True)  # 删除第二个 'Time' 列

# 获取表头
x_headers = Import_df.columns.tolist()

# 选择输入列（从第二列到第六列，控制和动力相关特征）
X = Import_df[['AntriebsmomentVA', 'AntriebsmomentHA', 'Lenkwinkel', 'BremsmomentVA', 'BremsmomentHA']].values

# 归一化输入数据
scaler_X = MinMaxScaler(feature_range=(0, 1))
X_scaled = scaler_X.fit_transform(X)

# 获取输出数据
Exoport_csv_file = './output.csv'  # 你可以根据需求选择正确的文件
Exoport_merged_data = pd.read_csv(Exoport_csv_file)

# 获取输出列头
y_headers = Exoport_merged_data.columns.tolist()

# 选择输出列（如车辆状态和行为相关特征）
y = Exoport_merged_data[['KS_X', 'KS_Y', 'Gierwinkel', 'Geschw_X', 'Geschw_Y', 'Giergeschwindigkeit']].values

# 归一化输出数据
scaler_y = MinMaxScaler(feature_range=(0, 1))
y_scaled = scaler_y.fit_transform(y)

# 将数据转换为LSTM所需的时间步格式
def create_dataset(X, y, time_step=5, forecast_horizon=1):
    X_data = []
    for i in range(len(X) - time_step - forecast_horizon + 1):
        input_x = X[i:(i + time_step)]  # 过去time_step个时间步的数据
        input_y = y[i + time_step:i + time_step + forecast_horizon]  # 对应的目标数据
        
        X_data.append(input_x)
    return np.array(X_data)


time_step = 5  # 输入时间步数
forecast_horizon = 1  # 预测1个未来时间步
X_data = create_dataset(X_scaled, y_scaled, time_step, forecast_horizon)

# 批量预测，开启 Keras 的进度条
X_data_reshaped = X_data.reshape(-1, time_step, X.shape[1])
#y_pred = model.predict(X_data_reshaped, verbose=1)  # verbose=1 显示进度条
# 将前五个时间步的预测值设置为实际值
#y_pred_with_actual = np.vstack((y_scaled[:5], y_pred))

# 初始化预测数组：前五步用真实值
y_pred = np.zeros_like(y_scaled)
y_pred[:5] = y_scaled[:5]
# 批量预测：从第6步开始，整体预测
batch_inputs = X_data_reshaped[5:]  # 批量输入数据
# 使用批量预测并添加进度条
batch_size = 64  # 批量大小，可根据显存/内存调整

for i in tqdm(range(0, len(batch_inputs), batch_size), desc="批量预测进度"):
    batch_seq = batch_inputs[i:i + batch_size]
    batch_pred = model.predict(batch_seq, verbose=0)
    
    # 存储预测结果
    y_pred[5 + i:5 + i + len(batch_pred)] = batch_pred

# 最终预测结果
y_pred_with_actual = y_pred

# 从第六步开始批量预测，添加 tqdm 进度条
#for i in tqdm(range(5, len(X_data_reshaped)), desc="批量预测进度"):
    # 构造输入：取前 time_step 步的数据作为输入
    #input_seq = X_data_reshaped[i - time_step + 1:i + 1]
    
    # 预测当前时间步
    #pred = model.predict(input_seq, verbose=0)
    
    # 存储预测结果
    #y_pred[i] = pred[-1]

# 反归一化预测结果
y_pred_reshaped = np.array(y_pred).reshape(-1, 6)
y_pred_rescaled = scaler_y.inverse_transform(y_pred_reshaped).reshape(len(y_pred), forecast_horizon, 6)


# 将预测结果转换为DataFrame并与输入数据合并
pred_df = pd.DataFrame(y_pred_rescaled.reshape(-1, 6))
pred_df.columns = y_headers[1:]  # 输出的列名

# 将预测结果与原始输入数据合并
merged_data = pd.concat([Import_df, pred_df], axis=1)

# 保存结果到CSV文件
merged_data.to_csv('pred_result.csv', index=False)

批量预测进度: 100%|██████████████████████████████████████████████████████████████████| 391/391 [00:19<00:00, 19.75it/s]
